In [235]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [236]:
int_reactions= list()
i= -1
n= 0
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    n= int((len(csvFile)-1)/2)

    flt= float(csvFile.iloc[n,  2])
    
    if v == 1 and csvFile.iloc[n,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[n,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR180tipp'

In [351]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= int(((len(csvFile)-1)/2)+1)

    while i3< len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4< int(((len(csvFile)-1)/2)):
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [238]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat above zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['2MAHMP',
 'AACPS8',
 'AACPS9',
 'ABTA',
 'ACCOAC',
 'ACONTa',
 'ACONTb',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'ACPPAT160',
 'ACPPAT161',
 'ACPPAT180',
 'ACPPAT181',
 'ALAALAr',
 'ALAR',
 'ADNCYC',
 'AP4AH',
 'AP4AS',
 'CMtex',
 'CD2tpp',
 'COBALT2tpp',
 'CS',
 'DASYN120',
 'DASYN140',
 'DASYN141',
 'DASYN160',
 'CU2abcpp',
 'DASYN161',
 'DASYN180',
 'DASYN181',
 'CU2tpp',
 'DOXRBCNtex',
 'ARGORNt7pp',
 'DTMPK',
 'EDA',
 'EDD',
 'DURADx',
 'ARGt3pp',
 'FACOAE60',
 'FBA3',
 'FHL',
 'FORtppi',
 'FUSAtex',
 'GK1',
 'G5SD',
 'G6PDH2r',
 'GLCDpp',
 'GLCNt2rpp',
 'GALUi',
 'GLNS',
 'GLU5K',
 'GF6PTA',
 'HYPOE',
 'HCO3E',
 'ICDHyr',
 'HEX4',
 'Kt3pp',
 'LCTStpp',
 'LYSt3pp',
 'MCOATA',
 'MALt3pp',
 'MAN1PT2',
 'MINCYCtex',
 'MELIBt2pp',
 'MELIBt3ipp',
 'MN2t3pp',
 'NADDP',
 'NADK',
 'NADPPPS',
 'GSPMDA',
 'GSPMDS',
 'NDPK2',
 'NDPK4',
 'NOVBCNtex',
 'NH4tpp',
 'NTD4',
 'NTD5',
 'GTPDPK',
 'O2tex',
 'P5CR',
 'GUACYC',
 'PGCD',
 'PDE1',
 'PDE4',
 'PDXPP',
 'PSERT',
 'PSP_L',
 'RBK',
 'RF

In [239]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat below zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


[]

In [240]:
i2= 0
mon_incr_above_zero= list()
mon_decr_above_zero= list()

if len(diff_indecr_above_zero)> 0:
    
    for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_above_zero)
set2= set(mon_decr_above_zero)
set3= set(mon_incr_above_zero)
inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_above_zero


['PSSA160',
 'NH4tex',
 'PRATPP',
 'SADT2',
 'LEUTAi',
 'KDOPS',
 'DM_5drib_c',
 'SDPDS',
 'THDPS',
 'MOAT',
 'CYSS',
 'IGPDH',
 'PPND',
 'ARGSL',
 'THRS',
 'UDCPDP',
 'PRAIi',
 'PE161abcpp',
 'ASPK',
 'HISTD',
 'UPP3S',
 'DAPDC',
 'SHKK',
 'MTHFR2',
 'PSSA161',
 'SERAT',
 'AGPAT161',
 'GLUTRR',
 'PRMICI',
 'UAPGR',
 'SULR',
 'ACHBS',
 'KARA2',
 'G1PACT',
 'TMDS',
 'U23GAAT',
 'UAGAAT',
 'A5PISO',
 'DHDPS',
 'USHD',
 'ARGSS',
 'UGMDDS',
 'MCTP1App',
 'DHQTi',
 'MPTG',
 'MOAT2',
 'G5SADs',
 'PSD161',
 'HMBS',
 'PRFGS',
 'PRASCSi',
 'ADSL1r',
 'DHDPRy',
 'FCLT',
 'ASPCT',
 'METAT',
 '3OAS140',
 'CHORM',
 'CYSTL',
 'PRAIS',
 'AIRC2',
 'OCBT',
 'PRAGSr',
 'ANPRT',
 'GMPS2',
 'SHSL1',
 'NNATr',
 'BPNT',
 'IG3PS',
 'SHK3Dr',
 'K2L4Aabctex',
 'KDOCT2',
 'UHGADA',
 'SO4tex',
 'METS',
 '5DOAN',
 'PAPPT3',
 'DHAD2',
 'LPADSS',
 'UPPDC1',
 'HISTP',
 'APG3PAT161',
 'G1SAT',
 'HSTPT',
 'CHORS',
 'TDSK',
 'ACLS',
 'APG3PAT160',
 'IGPS',
 '3OAR140',
 'UAGPT3',
 'GLUPRT',
 'ACGK',
 'CTPS2',
 'PSCVT',


In [241]:
w= list()
w2= list()
i2=0

if len(inANDdecr_above_zero)> 0:
    
    while i2 < len(inANDdecr_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['PSSA160',
 'PRATPP',
 'SADT2',
 'KDOPS',
 'DM_5drib_c',
 'MOAT',
 'CYSS',
 'IGPDH',
 'PPND',
 'ARGSL',
 'UDCPDP',
 'PRAIi',
 'PE161abcpp',
 'HISTD',
 'UPP3S',
 'MTHFR2',
 'PSSA161',
 'SERAT',
 'AGPAT161',
 'GLUTRR',
 'PRMICI',
 'UAPGR',
 'SULR',
 'ACHBS',
 'KARA2',
 'G1PACT',
 'TMDS',
 'U23GAAT',
 'UAGAAT',
 'A5PISO',
 'USHD',
 'ARGSS',
 'UGMDDS',
 'MCTP1App',
 'MPTG',
 'MOAT2',
 'G5SADs',
 'PSD161',
 'HMBS',
 'ADSL1r',
 'FCLT',
 'METAT',
 '3OAS140',
 'CYSTL',
 'OCBT',
 'ANPRT',
 'GMPS2',
 'SHSL1',
 'NNATr',
 'BPNT',
 'IG3PS',
 'K2L4Aabctex',
 'KDOCT2',
 'UHGADA',
 'SO4tex',
 'METS',
 '5DOAN',
 'PAPPT3',
 'DHAD2',
 'LPADSS',
 'UPPDC1',
 'HISTP',
 'APG3PAT161',
 'G1SAT',
 'HSTPT',
 'TDSK',
 'APG3PAT160',
 'IGPS',
 '3OAR140',
 'UAGPT3',
 'ACGK',
 'CTPS2',
 'AGPAT160',
 'PPNDH',
 'THRD_L',
 'DHFR',
 'ACGS',
 'UAAGDS',
 'K2L4Aabcpp',
 'ADSS',
 'UAGDP',
 'ACODA',
 'UAMAGS',
 'GLUTRS',
 'ANS',
 'ATPPRT',
 'KDOPP',
 'UAMAS',
 'HSST',
 'NADS1',
 'UAGCVT',
 'PPBNGS',
 'ADSK',
 'PRAMPC',
 'PE1

In [242]:
set1= set(inANDdecr_above_zero)
set2= set(w)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_decr_above_zero= list(mon_decr_above_zero) + list(w)


set1= set(inANDdecr_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_incr_above_zero= list(mon_incr_above_zero) + list(w2)


In [243]:
i2= 0
mon_incr_below_zero= list()
mon_decr_below_zero= list()

if len(diff_indecr_below_zero)> 0:
    
    for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero)
set2= set(mon_decr_below_zero)
set3= set(mon_incr_below_zero)
inANDdecr_below_zero= list(set1-set2-set3)
inANDdecr_below_zero


[]

In [244]:
w= list()
w2= list()
i2=0

if len(inANDdecr_below_zero)> 0:
    
    while i2 < len(inANDdecr_below_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [245]:
w2

[]

In [246]:
set1= set(inANDdecr_below_zero)
set2= set(w)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_decr_below_zero= list(mon_decr_below_zero) + list(w)


set1= set(inANDdecr_below_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_incr_below_zero= list(mon_incr_below_zero) + list(w2)


In [247]:
i2= 0
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

if len(indecr_and_stable_above_zero)> 0:
    
    for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['ALAALAr',
 'PGCD',
 'DTMPK',
 'NDPK2',
 'ACPPAT161',
 'HCO3E',
 'DASYN161',
 'P5CR',
 'PSERT',
 'CU2tpp',
 'PSP_L',
 'DASYN160',
 'ACONTa',
 'GLNS',
 'MCOATA',
 'CS',
 'ALAR',
 'TMPK',
 'ACONTb',
 'COBALT2tpp',
 'GK1',
 'NADK',
 'ACCOAC',
 'UMPK',
 'ACPPAT160',
 'GF6PTA',
 'NH4tpp',
 'ICDHyr',
 'PMPK',
 'NDPK4']

In [248]:
w= list()
w2= list()
i2=0

if len(inANDdecr_and_stable_above_zero)> 0:
    
    while i2 < len(inANDdecr_and_stable_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['ALAALAr',
 'DTMPK',
 'ACPPAT161',
 'DASYN161',
 'P5CR',
 'CU2tpp',
 'DASYN160',
 'MCOATA',
 'ALAR',
 'TMPK',
 'COBALT2tpp',
 'GK1',
 'NADK',
 'ACCOAC',
 'ACPPAT160',
 'GF6PTA',
 'PMPK',
 'NDPK4']

In [249]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


set1= set(inANDdecr_and_stable_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_incr_and_stable_above_zero= list(mon_incr_and_stable_above_zero) + list(w2)


In [250]:
i2= 0
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

if len(indecr_and_stable_below_zero)> 0:
    
    for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero.append(r_id)

        i2= i2 + 1

set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


[]

In [251]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero):       #this code finds reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

            
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [252]:
set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero= list(set4)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [253]:
i= 0
over_below_zero_and_under_above_zero= list()
under_below_zero_and_over_above_zero= list()
over_below_zero_and_over_above_zero= list()
under_below_zero_and_under_above_zero= list()
r_id = a.reactions[0]

for r_id in a.reactions:    #separates reactions based on whether they have less than 30 or not values both in the above and the below zero interval
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < int(((len(csvFile)-1)/2)+1):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= int((len(csvFile)-1)/2)
    l4= list()
    
    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2)>30 or len(csvFile2)==30) and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
        
        over_below_zero_and_under_above_zero.append(r_id)
        
    if len(csvFile2)<30 and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        under_below_zero_and_over_above_zero.append(r_id)
    
    if (len(csvFile2)>30 or len(csvFile2)==30) and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        over_below_zero_and_over_above_zero.append(r_id)
        
    if len(csvFile2)<30 and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
    
        under_below_zero_and_under_above_zero.append(r_id)
    
    
    i= i+1
    
over_below_zero_and_under_above_zero


['EX_cmp_e',
 'EX_cpgn_e',
 'EX_cpgn_un_e',
 'EX_csn_e',
 'EX_cu_e',
 'EX_glcr_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_gtp_e',
 'EX_h2_e',
 'EX_23camp_e',
 'EX_23ccmp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_23dappa_e',
 'EX_26dap__M_e',
 'EX_2ddglcn_e',
 'EX_hdca_e',
 'EX_3amp_e',
 'EX_hdcea_e',
 'EX_3cmp_e',
 'EX_cynt_e',
 'EX_3gmp_e',
 'EX_3hcinnm_e',
 'EX_3hpppn_e',
 'EX_3ump_e',
 'EX_LalaDglu_e',
 'EX_cytd_e',
 'EX_LalaLglu_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_isetac_e',
 'EX_acgal1p_e',
 'EX_acgam_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acmum_e',
 'EX_acnam_e',
 'EX_lcts_e',
 'EX_dca_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_lyx__L_e',
 'EX_mal__D_e',
 'EX_ddca_e',
 'EX_malt_e',
 'EX_all__D_e',
 'EX_dgmp_e',
 'EX_man_e',
 'EX_man6p_e',
 'EX_manglyc_e',
 'EX_amp_e',
 'EX_anhgm_e',
 'EX_arab__L_e',
 'EX_arbtn_e',
 'EX_melib_e',
 'EX_arbtn_fe3_e',
 'EX_dgsn_e',
 'EX_ascb__L_

In [254]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_diff_indecr_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
flat_below_zero_and_one_to_three_data_point_above_zero= list()
diff_indecr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(over_below_zero_and_under_above_zero)> 0:

    for r_id in over_below_zero_and_under_above_zero:    #separates over_below_zero_and_under_above_zero reactions in more specific subcategories
    
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and len(csvFile3)<4:
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 3 and count2< len(csvFile3)-4):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 4:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and len(csvFile3)<4:
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
        elif count< 0.05*len(csvFile2) and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 3 and count2< len(csvFile3)-4):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 4:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif count< 0.05*len(csvFile2) and (count2> len(csvFile3)-4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2> 2 and len(csvFile3) < 4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [255]:
i=0
one_to_three_data_point_below_zero_and_diff_indecr_above_zero= list()
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

if len(under_below_zero_and_over_above_zero)> 0:

    for r_id in under_below_zero_and_over_above_zero:    #separates under_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 4 and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

            
    
        elif len(csvFile2)<4 and count2< 0.05*len(csvFile3):
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [256]:
i=0

if len(over_below_zero_and_over_above_zero)> 0:

    for r_id in over_below_zero_and_over_above_zero:    #separates over_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count<0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ACALD',
 'ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CITL',
 'CMtpp',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'ARBt2rpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx',
 'FADRx2',
 'FBA',
 'FBP',
 'FE2t2pp',
 'ASNS2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'G6PP',
 'GLCptspp',
 'GLCt2pp',
 'ASPT',
 'GLYCDx',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GUAt2pp',
 'GLYCLTDx',
 'GLYCLTDy',
 'IDOND2',
 'GLYK',
 'HPYRRy',
 'Kt2pp',
 'LCTSt3ipp',
 'LYSt2pp',
 'MACPD',
 'MALS',
 'MALt2_2pp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'ME2',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN2tpp',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'OAADC',
 'PFL',
 'PTRCt2pp',
 'RFAMPtpp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'SSALx',
 'SSALy',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCCt2_3pp',
 'SUCMALtpp',
 'TALA',
 'THD2pp',
 'THRA2',
 'THRt4pp',
 'TRE6PH',
 'TTRCYCtpp',
 'XANt2pp',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp

In [257]:
i=0
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero= list()

if len(under_below_zero_and_under_above_zero)> 0:
    
    for r_id in under_below_zero_and_under_above_zero:    #separates under_below_zero_and_under_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and len(csvFile3)<4:
            one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

        elif len(csvFile2)<4 and (count2> 3 and count2< len(csvFile3)-4):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 3 and count2< len(csvFile3)-4):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 4 and (count2> 3 and count2< len(csvFile3)-4):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 2 and len(csvFile3) < 4):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 1 and len(csvFile3) < 3):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> 2 and len(csvFile3) < 4):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<4 and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            flat_below_zero_and_flat_above_zero.append(r_id)

        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and len(csvFile3) < 4):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and count2< 4:
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count> 3 and count< len(csvFile2)-4) and count2< 4:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 4 and count2< 4:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 4) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 4:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ACALD',
 'ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CITL',
 'CMtpp',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'ARBt2rpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx',
 'FADRx2',
 'FBA',
 'FBP',
 'FE2t2pp',
 'ASNS2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'G6PP',
 'GLCptspp',
 'GLCt2pp',
 'ASPT',
 'GLYCDx',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GUAt2pp',
 'GLYCLTDx',
 'GLYCLTDy',
 'IDOND2',
 'GLYK',
 'HPYRRy',
 'Kt2pp',
 'LCTSt3ipp',
 'LYSt2pp',
 'MACPD',
 'MALS',
 'MALt2_2pp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'ME2',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN2tpp',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'OAADC',
 'PFL',
 'PTRCt2pp',
 'RFAMPtpp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'SSALx',
 'SSALy',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCCt2_3pp',
 'SUCMALtpp',
 'TALA',
 'THD2pp',
 'THRA2',
 'THRt4pp',
 'TRE6PH',
 'TTRCYCtpp',
 'XANt2pp',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp

In [258]:
flat_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero
indecr_and_stable_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_diff_indecr_above_zero
diff_indecr_below_zero_and_indecr_and_stable_above_zero
diff_indecr_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_indecr_and_stable_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_flat_above_zero


indecr_and_stable_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero


['CRNDt2rpp', 'FRD2', 'Htex', 'NADH17pp', 'NADH18pp', 'PTAr']

In [259]:
len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero_and_flat_above_zero)+len(one)+len(zero)+len(flat_below_zero_and_flat_above_zero)+len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_diff_indecr_above_zero)+len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_diff_indecr_above_zero)+len(flat_below_zero_and_indecr_and_stable_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_flat_above_zero)+len(one_at_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


2726

In [260]:
lt= list(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero_and_flat_above_zero)+list(one)+list(zero)+list(flat_below_zero_and_flat_above_zero)+list(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_diff_indecr_above_zero)+list(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+list(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_diff_indecr_above_zero)+list(flat_below_zero_and_indecr_and_stable_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_flat_above_zero)+list(one_at_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)


In [261]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [262]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [263]:
i2= 0
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

if len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:                    #this code finds which of the indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

set1= set(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set3= set(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


['G3PD2', 'ASPTA', 'EX_isetac_e']

In [264]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
    
        i2= i2+1

w3


[]

In [265]:
set1= set(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set4)

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [266]:
i2= 0
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                  #this code finds which of the indecr_and_stable_below_zero_and_diff_indecr_above_zero reactions have an increasing trend and which a decreasing trend above zero
    
        r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [267]:
w3= list()
w4= list()
i2=0

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable above zero
    
        i= int((len(csvFile)-1)/2)
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        
        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)
        
            
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [268]:
set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set4)

indecr_and_stable_below_zero_and_mon_decr_above_zero= list(indecr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
indecr_and_stable_below_zero_and_mon_incr_above_zero= list(indecr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [269]:
i2= 0
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
incr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                #this code finds which of the indecr_and_stable_below_zero_and_mon_decr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['RPI', 'ACOAD1f', 'PGK', 'ACOAD2f', 'CO2tpp', 'PGM', 'GLUDy']

In [270]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [271]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero) + list(w4)


In [272]:
i2= 0
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
incr_and_stable_below_zero_and_mon_incr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_incr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:              #this code finds which of the indecr_and_stable_below_zero_and_mon_incr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [273]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [274]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [275]:
i2= 0
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:           #this code finds which of the indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)


        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [276]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [277]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set4)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [278]:
i2= 0
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

if len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)
set2= set(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)
set3= set(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero


['FE2t3pp']

In [279]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['FE2t3pp']

In [280]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero) + list(w4)


In [281]:
i2= 0
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

if len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_diff_indecr_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)
set2= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
set3= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero


[]

In [282]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [283]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set4)

one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list(one_to_three_data_point_below_zero_and_mon_incr_above_zero) + list(w4)


In [284]:
i2= 0
diff_indecr_below_zero_and_mon_incr_above_zero= list()
diff_indecr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_diff_indecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                diff_indecr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                diff_indecr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_diff_indecr_above_zero)
set2= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set3= set(diff_indecr_below_zero_and_mon_incr_above_zero)
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
diff_indecr_below_zero_and_inANDdecr_above_zero


['ALAALAabcpp',
 'ECOAH3',
 'MLDCP1Bpp',
 'EX_co2_e',
 'GHMT2r',
 'MLDEP2pp',
 'MLDCP2Bpp',
 'MDDCP1pp',
 '3PEPTabcpp',
 'CPPPGO',
 'AGMt2pp',
 'TRPAS2',
 'SPMDabcpp']

In [285]:
w3= list()
w4= list()
i2=0

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(diff_indecr_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['ALAALAabcpp',
 'ECOAH3',
 'MLDCP1Bpp',
 'MLDEP2pp',
 'MLDCP2Bpp',
 'MDDCP1pp',
 '3PEPTabcpp',
 'CPPPGO',
 'AGMt2pp',
 'TRPAS2',
 'SPMDabcpp']

In [286]:
w4

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set4)

diff_indecr_below_zero_and_mon_decr_above_zero= list(diff_indecr_below_zero_and_mon_decr_above_zero) + list(w3)
diff_indecr_below_zero_and_mon_incr_above_zero= list(diff_indecr_below_zero_and_mon_incr_above_zero) + list(w4)


In [287]:
i2= 0
mon_decr_below_zero_and_mon_decr_above_zero= list()
mon_incr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_decr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_above_zero)
set3= set(mon_incr_below_zero_and_mon_decr_above_zero)
inANDdecr_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_decr_above_zero


['EX_quin_e', 'CO2tex', 'EX_ura_e', 'EX_val__L_e']

In [288]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_decr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

        
        
w3


[]

In [289]:
w4

['EX_quin_e', 'EX_ura_e']

In [290]:
w4

set1= set(inANDdecr_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_mon_decr_above_zero= list(set4)

mon_decr_below_zero_and_mon_decr_above_zero= list(mon_decr_below_zero_and_mon_decr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_decr_above_zero= list(mon_incr_below_zero_and_mon_decr_above_zero) + list(w4)


In [291]:
i2= 0
mon_decr_below_zero_and_mon_incr_above_zero= list()
mon_incr_below_zero_and_mon_incr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_incr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_incr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_incr_above_zero)
set2= set(mon_decr_below_zero_and_mon_incr_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_above_zero)
inANDdecr_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_incr_above_zero


[]

In [292]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_incr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [293]:
w4

set1= set(inANDdecr_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_mon_incr_above_zero= list(set4)

mon_decr_below_zero_and_mon_incr_above_zero= list(mon_decr_below_zero_and_mon_incr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_above_zero= list(mon_incr_below_zero_and_mon_incr_above_zero) + list(w4)


In [294]:
i2= 0
mon_decr_below_zero_and_inANDdecr_above_zero= list()
mon_incr_below_zero_and_inANDdecr_above_zero= list()

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_inANDdecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_inANDdecr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(mon_decr_below_zero_and_inANDdecr_above_zero)
set3= set(mon_incr_below_zero_and_inANDdecr_above_zero)
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_above_zero


[]

In [295]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [296]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set4)

mon_decr_below_zero_and_inANDdecr_above_zero= list(mon_decr_below_zero_and_inANDdecr_above_zero) + list(w3)
mon_incr_below_zero_and_inANDdecr_above_zero= list(mon_incr_below_zero_and_inANDdecr_above_zero) + list(w4)


In [297]:
i2= 0
mon_decr_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_below_zero_and_indecr_and_stable_above_zero= list()

if len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = diff_indecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_indecr_and_stable_above_zero


[]

In [298]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [299]:
w4

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set4)

mon_decr_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [300]:
i2= 0
inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [301]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [302]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [303]:
i2= 0
mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [304]:
w3= list()
w4= list()
i2=0

if len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [305]:
w4

set1= set(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [306]:
i2= 0
mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero


['ECOAH4',
 'ECOAH2',
 'GNK',
 'GAPD',
 'FUM',
 'MTHFD',
 'ENO',
 'TMDK1',
 'TPI',
 'MTHFC',
 'ECOAH1',
 'CYSabcpp',
 'MDH']

In [307]:
w3= list()
w4= list()
i2=0

if len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['ECOAH4', 'ECOAH2', 'GNK', 'TMDK1', 'ECOAH1', 'CYSabcpp']

In [308]:
w4

set1= set(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [309]:
i2= 0
mon_decr_below_zero_and_one_to_three_data_point_above_zero= list()
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_one_to_three_data_point_above_zero:
    
        r_id = diff_indecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(mon_decr_below_zero_and_one_to_three_data_point_above_zero)
set3= set(mon_incr_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


['ACOTA',
 'KARA1',
 'AIRC3',
 'IMPC',
 'EX_so4_e',
 'IPPMIa',
 'DHORTS',
 'SDPTA',
 'ILETA',
 'AGPR',
 'TYRTA',
 'EX_mso3_e',
 'EX_nh4_e',
 'PHETA1',
 'EX_26dap__M_e',
 'EX_pi_e',
 'HSDy',
 'IPPMIb',
 'EX_skm_e',
 'ORPT',
 'EX_orot_e',
 'ASAD']

In [310]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [311]:
w4

set1= set(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set4)

mon_decr_below_zero_and_one_to_three_data_point_above_zero= list(mon_decr_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list(mon_incr_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [312]:
i2= 0
flat_below_zero_and_mon_decr_above_zero= list()
flat_below_zero_and_mon_incr_above_zero= list()

if len(flat_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in flat_below_zero_and_diff_indecr_above_zero:
    
        r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [313]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [314]:
w4

set1= set(flat_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
flat_below_zero_and_inANDdecr_above_zero= list(set4)

flat_below_zero_and_mon_decr_above_zero= list(flat_below_zero_and_mon_decr_above_zero) + list(w3)
flat_below_zero_and_mon_incr_above_zero= list(flat_below_zero_and_mon_incr_above_zero) + list(w4)


In [315]:
i2= 0
flat_below_zero_and_mon_decr_and_stable_above_zero= list()
flat_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(flat_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in flat_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_mon_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [316]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [317]:
w4

set1= set(flat_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

flat_below_zero_and_mon_decr_and_stable_above_zero= list(flat_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
flat_below_zero_and_mon_incr_and_stable_above_zero= list(flat_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [318]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [319]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [320]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_flat_above_zero= list(set4)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [321]:
i2= 0
mon_decr_and_stable_below_zero_and_flat_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

if len(indecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_flat_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_flat_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [322]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [323]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_flat_above_zero= list(mon_decr_and_stable_below_zero_and_flat_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_flat_above_zero= list(mon_incr_and_stable_below_zero_and_flat_above_zero) + list(w4)


In [324]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [325]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [326]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_below_zero_and_flat_above_zero= list(set4)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [327]:
i2= 0
mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()

if len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ME2', 'THD2pp', 'FLDR2', 'PPCK', 'PPS', 'CITL', 'FBP', 'GLYCDx']

In [328]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [329]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [330]:
i2= 0
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [331]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [332]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [333]:
i2= 0
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [334]:
w3= list()
w4= list()
i2=0

if len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [335]:
w4

set1= set(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [336]:
i2= 0
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [337]:
w3= list()
w4= list()
i2=0

if len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

        
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [338]:
w4

set1= set(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set4)

mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [339]:
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero

['EX_no_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_tmao_e',
 '2DGULRGx',
 'ADK4',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CTBTt2rpp',
 'CUt3',
 'DOGULNR',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LEUt2rpp',
 'LIPOCT',
 'MLTP1',
 'MLTP2',
 'MLTP3',
 'MG2t3_2pp',
 'NODOx',
 'OMMBLHX',
 'OMPHHX',
 'OPHHX',
 'PGMT',
 'PTRCORNt7pp',
 'SUCFUMtpp',
 'SUCTARTtpp',
 'TARTD',
 'TREptspp',
 'PMANM',
 'XANtpp',
 'VALt2rpp',
 'MECDPDH2',
 'XYLt2pp',
 '2DGLCNRx',
 'TARTDC',
 'G3PD2']

In [340]:
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

decr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_mon_incr_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero

decr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_mon_incr_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_above_zero
mon_decr_below_zero_and_mon_decr_above_zero
mon_incr_below_zero_and_mon_decr_above_zero

inANDdecr_below_zero_and_mon_incr_above_zero
mon_decr_below_zero_and_mon_incr_above_zero
mon_incr_below_zero_and_mon_incr_above_zero

inANDdecr_below_zero_and_inANDdecr_above_zero
mon_decr_below_zero_and_inANDdecr_above_zero
mon_incr_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_below_zero_and_one_to_three_data_point_above_zero
mon_decr_below_zero_and_one_to_three_data_point_above_zero
mon_incr_below_zero_and_one_to_three_data_point_above_zero

flat_below_zero_and_inANDdecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero

flat_below_zero_and_inANDdecr_and_stable_above_zero
flat_below_zero_and_mon_decr_and_stable_above_zero
flat_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_and_stable_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero_and_flat_above_zero
mon_incr_and_stable_below_zero_and_flat_above_zero

inANDdecr_below_zero_and_flat_above_zero
mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero

inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero


['EX_cpgn_un_e',
 'EX_cu_e',
 'EX_gtp_e',
 'EX_acgal1p_e',
 'EX_arbtn_e',
 'EX_minohp_e',
 'EX_dopa_e',
 'EX_xmp_e',
 'EX_peamn_e',
 'EX_tym_e',
 'EX_tyrp_e',
 '2DGULRy',
 'EX_fe3hox_un_e',
 'EX_fecrm_un_e',
 'EX_feoxam_un_e',
 'ACt2rpp',
 'ALAt2pp_copy2',
 'ALDD19xr_copy1',
 'AMALT2',
 'AMALT3',
 'AMALT4',
 'CAt6pp',
 'CRNCAL2',
 'CRNt2rpp',
 'CTBTCAL2',
 'CU1Opp',
 'DHNPA2r_copy1',
 'DKGLCNR2x',
 'DMPPS',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DSBAO1',
 'DSBAO2',
 'GLYCLTt2rpp',
 'HYD1pp',
 'HYD2pp',
 'HYD3pp',
 'IPDPS',
 'HPYRI',
 'GLYt2pp_copy2',
 'MECDPDH5',
 'NO3R2pp',
 'NHFRBO',
 'NMNt7pp',
 'NO3R1pp',
 'OCTNLL',
 'PDX5PO2',
 'PDX5POi',
 'PROt2rpp',
 'QMO2',
 'QMO3',
 'SERt2rpp',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'TREHpp',
 'URAt2pp_copy2',
 'NPHS',
 'DARBt2rpp_copy1']

In [341]:
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

['EX_csn_e',
 'EX_cynt_e',
 'EX_o2_e',
 '2DGULRGy',
 '2DGULRx',
 'BETALDHx',
 'BSORy',
 'DKGLCNR2y',
 'FUMt2_3pp',
 'NODOy',
 'NO3R1bpp',
 'NO3R2bpp',
 'OMMBLHX3',
 'OMPHHX3',
 'OPHHX3',
 '2DGLCNRy',
 'DOGULND']

In [342]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


2726

In [343]:
lt= list(flat_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ list(flat_below_zero_and_one_to_three_data_point_above_zero)+ list(one_at_zero)+list(one)+list(zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)+list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_above_zero)+list(mon_decr_below_zero_and_mon_decr_above_zero)+list(mon_incr_below_zero_and_mon_decr_above_zero)+list(inANDdecr_below_zero_and_mon_incr_above_zero)+list(mon_decr_below_zero_and_mon_incr_above_zero)+list(mon_incr_below_zero_and_mon_incr_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_above_zero)+list(mon_decr_below_zero_and_inANDdecr_above_zero)+list(mon_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_mon_decr_and_stable_above_zero)+list(flat_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_flat_above_zero)+list(mon_decr_and_stable_below_zero_and_flat_above_zero)+list(mon_incr_and_stable_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


In [344]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [345]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [346]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)


2726

In [347]:
len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


536

In [348]:
len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)


536